In [ ]:
import ee
import pandas as pd

# Initialize the Earth Engine library
ee.Initialize(project='pranavkoka123')

# Define a simple region of interest (a small area)
geometry = ee.Geometry.Rectangle([79.0333, 30.7333, 79.0667, 30.8000])

# Get a single Landsat 8 image for a specific date
image = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA') \
            .filterDate('2015-01-01', '2015-01-31') \
            .filterBounds(geometry) \
            .first()

# Calculate NDSI function
def calculate_ndsi(image):
    # NDSI = (Green - SWIR) / (Green + SWIR)
    green = image.select('B3')  # Green band
    swir = image.select('B6')  # SWIR band
    ndsi = green.subtract(swir).divide(green.add(swir)).rename('NDSI')
    return image.addBands(ndsi)

# Apply NDSI calculation
image_ndsi = calculate_ndsi(image)

# Apply threshold to NDSI values
def apply_ndsi_threshold(image):
    # NDSI threshold: set values less than 0.1 to 0
    ndsi = image.select('NDSI')
    ndsi_thresholded = ndsi.where(ndsi.lt(0.1), 0).rename('NDSI_thresholded')
    return image.addBands(ndsi_thresholded)

# Apply the threshold to the NDSI
image_ndsi_thresholded = apply_ndsi_threshold(image_ndsi)

# Function to extract snow/ice confidence from QA band (Bits 12-13) and snow confidence (Bit 5)
def extract_confidence_bits(image):
    qa = image.select('QA_PIXEL')  # Assuming QA_PIXEL contains the quality band

    # Extract Snow/Ice Confidence (Bits 12-13)
    snow_ice_confidence = qa.rightShift(12).bitwiseAnd(3).rename('Snow_Ice_Confidence')
    
    # Extract Snow Confidence (Bit 5)
    snow_confidence = qa.rightShift(5).bitwiseAnd(1).rename('Snow_Confidence')

    return image.addBands([snow_ice_confidence, snow_confidence])

# Apply the bit extraction
image_with_confidence = extract_confidence_bits(image_ndsi_thresholded)

# Reduce the region to get pixel values for NDSI (both thresholded and original), Snow/Ice Confidence, and Snow Confidence
def get_pixel_values(image, geometry):
    # Create a dictionary of band values
    reduced_image = image.reduceRegion(
        reducer=ee.Reducer.toList(), 
        geometry=geometry, 
        scale=30
    )
    return reduced_image

# Get the pixel values from the image
pixel_values = get_pixel_values(image_with_confidence, geometry)

# Convert the data into a DataFrame
ndsi_values = pixel_values.get('NDSI').getInfo()  # Original NDSI values
ndsi_thresholded_values = pixel_values.get('NDSI_thresholded').getInfo()  # Thresholded NDSI values
snow_ice_conf_values = pixel_values.get('Snow_Ice_Confidence').getInfo()
snow_conf_values = pixel_values.get('Snow_Confidence').getInfo()

# Create the DataFrame
df = pd.DataFrame({
    'NDSI': ndsi_values,  # Original NDSI
    'NDSI_thresholded': ndsi_thresholded_values,  # Thresholded NDSI
    'Snow/Ice Confidence': snow_ice_conf_values,
    'Snow Confidence': snow_conf_values
})

# Display the DataFrame
print(df)
